In [29]:
import dash
from dash import dcc
from dash import html
from dash import dash_table
from dash import Dash, _dash_renderer
from dash import callback
from dash.dependencies import Input, Output, ClientsideFunction
from jupyter_dash import JupyterDash

import numpy as np
import pandas as pd
import pathlib
import plotly.express as px
import dash_mantine_components as dmc
_dash_renderer._set_react_version("18.2.0")

full_champion_data = pd.read_csv('csv/full_champion_data.csv', index_col=0)
champion_role_winrate = pd.read_csv('csv/champion_role_winrate.csv', index_col=0)
champion_role_pickrate = pd.read_csv('csv/champion_role_pickrate.csv', index_col=0)
game_duration = pd.read_csv('csv/game_duration.csv', index_col=0)
game_time_graph = pd.read_csv('csv/game_time_graph.csv', index_col=0)
winrate_objective = pd.read_csv('csv/winrate_objective.csv', index_col=0)

winrate_objective_color = ["#4B0082",  "#0000FF", "#008080", "#008000", " #8b0000","#FFA500",  "#FF0000",  "#808080", "#000000"]

In [75]:
app = dash.Dash(
    __name__,
    meta_tags=[{"name": "viewport", "content": "width=device-width, initial-scale=1"}],
    external_stylesheets=dmc.styles.ALL
)
app.title = "LOL Analysis Project Data Visualization"

server = app.server
app.config.suppress_callback_exceptions = True

app.layout = dmc.MantineProvider(
    forceColorScheme="light",
    children=[
        dmc.Title("League of Legends Data Visualization and Analysis", id='dashboard-title', size="h3", style={"textAlign": "center"}),
        dmc.Divider(variant="solid", size="md", style={"marginTop": "10px", "marginBottom": "10px"}),
        dmc.Grid([
            dmc.GridCol([
                dmc.Title("Winrate by Objectives", id="winrate-objective-title", size="h5", style={"fontWeight": "bold", "fontStyle": "italic", "textDecoration": "underline"}),
                dmc.RadioGroup(
                    id='winrate-objective-radio', 
                    value=winrate_objective.columns[4], 
                    size='sm',
                    children=
                        dmc.Group(
                            children=[dmc.GridCol(dmc.Radio(i, value=i, color=winrate_objective_color[winrate_objective.columns.get_loc(i) % len(winrate_objective_color)]), span=2) for i in winrate_objective.columns]), 
                            style={"flexWrap": "wrap", "width": "100%"},
                        ),
                dash_table.DataTable(
                    id='winrate-objective-table', 
                    data=winrate_objective.reset_index().to_dict('records'), 
                    style_table={'overflowX': 'auto'},
                    style_cell={ 'textAlign': 'center', "padding": "10px"},
                    style_header={'fontWeight': 'bold','textAlign': 'center'}
                )
            ], span=5),
            dmc.GridCol([
                dmc.LineChart(
                    id='winrate-objective-linechart',
                    h=300,
                    data=winrate_objective.reset_index().to_dict('records'),
                    dataKey="index",
                    series=[
                        {"name": objective, 
                         "color": winrate_objective_color[i % len(winrate_objective_color)],
                        } 
                         for i, objective in enumerate(winrate_objective.columns)
                    ],
                    curveType="linear",
                    tickLine="xy",
                    xAxisLabel="Rank",
                    yAxisLabel="Winrate (%)",
                )
            ], span=7, style={"paddingTop": "130px"})
        ]),

        dmc.Divider(variant="solid", size="md", style={"marginTop": "10px", "marginBottom": "10px"}),
        dmc.Grid([
            dmc.GridCol([
                dmc.Title("Winrate by Roles", id="winrate-roles-title", size="h5", style={"fontWeight": "bold", "fontStyle": "italic", "textDecoration": "underline"})
            ]),
        ]),
    ]
)

@app.callback(
    Output("winrate-objective-linechart", "series"),
    Input("winrate-objective-radio", "value"),
)
def update_chart(objective):
    return [
        {
            "name": objective, "color": winrate_objective_color[winrate_objective.columns.get_loc(objective) % len(winrate_objective_color)],
        }
    ]

if __name__ == "__main__":
    app.run_server(mode="external", debug=True, port=8050, host="127.0.0.1")